## Importing Libraries and Initial Setup

The following libraries and modules are imported and initialized for text preprocessing and feature extraction:

`pandas`: Library for data manipulation and analysis.

`string`: Contains common string operations and constants.

`nltk`: The Natural Language Toolkit, a comprehensive library for natural language processing. Here, it is used for:

`nltk.tokenize.word_tokenize`: Tokenizes sentences into individual words (tokens).

`nltk.corpus.stopwords`: Provides a set of common stopwords in multiple languages. These stopwords are used for filtering irrelevant words (e.g., "the", "is") from the text.

`spacy`: Library for advanced natural language processing.

`sklearn.feature_extraction.text.TfidfVectorizer`: For converting text to TF-IDF features.

NLTK data (`punkt` and `stopwords`) is downloaded for tokenization and filtering out common stop words.

The `en_core_web_trf` model from spaCy is loaded for processing text with transformers specifically for lemmatization.

In [91]:
import pandas as pd
import string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import spacy
import json
from sklearn.feature_extraction.text import TfidfVectorizer
import stopwordsiso as stopwords
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

nlp = spacy.load("en_core_web_trf")

[nltk_data] Downloading package punkt to C:\Users\Lord
[nltk_data]     Allain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to C:\Users\Lord
[nltk_data]     Allain\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to C:\Users\Lord
[nltk_data]     Allain\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Filipino Stopwords

The following list contains commonly used Filipino stopwords. These are words that are typically filtered out in text processing as they do not carry significant meaning in the context of text analysis.

These stopwords include common words such as articles, prepositions, and conjunctions that are usually removed during text preprocessing to focus on more meaningful words in text analysis.

In [92]:
filipino_stopwords = stopwords.stopwords('tl')  

## Function: `load_dataset`

The `load_dataset` function is used to load a dataset from a CSV file. It handles various potential errors during the file loading process.

### Code

In [93]:
def load_dataset(file_path):
    try:
        data = pd.read_csv(file_path)
        print("Dataset loaded successfully!")
        return data
    except FileNotFoundError:
        print("File not found. Please check the file path.")
    except pd.errors.EmptyDataError:
        print("File is empty. Please check the file content.")
    except pd.errors.ParserError:
        print("Error parsing file. Please check the file format.")
    except Exception as e:
        print(f"An error occurred: {e}")

## Loading Dataset and Extracting Sentences

The following code snippet demonstrates how to load a dataset from a CSV file and extract sentences from it.

### Code

In [94]:
file_path = '../data/training_data_samples60.csv'
data = load_dataset(file_path)

sentences = data['sentence'].tolist()

Dataset loaded successfully!


## Function: `format_list_as_string`

The `format_list_as_string` function converts a list of tokens into a formatted string representation.

In text preprocessing, tokens are often stored as lists of words after steps like tokenization, stopword removal, and lemmatization. These lists can sometimes be difficult to read or display, especially when reviewing or debugging processed data.

this function ensures that these lists are formatted neatly when printed.

### Code

In [95]:
def format_list_as_string(token_list):
    return str(token_list).replace("'", '"')

## Function: `print_table`

The `print_table` function displays a formatted table using the `rich` library, which provides enhanced terminal output for data visualization.

### Code

In [96]:
def print_table(data, title="Table", num_samples=60):
    from rich.console import Console
    from rich.table import Table
    
    table = Table(title=title)
    
    for col in data.columns:
        table.add_column(col)

    for _, row in data.head(num_samples).iterrows():
        formatted_row = [format_list_as_string(row[col]) if isinstance(row[col], list) else row[col] for col in data.columns]
        table.add_row(*map(str, formatted_row))
    
    console = Console()
    console.print(table)

print_table(data, title="Original Data")

                                                   Original Data                                                   
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ "Your tibialis anterior is damaged."                                                                  │ fear    │
│ "Sadly, he's in the West."                                                                            │ sadness │
│ "Sorry."                                                                                              │ sadness │
│ "It’s just a joke."                                                                                   │ joy     │
│ "It's nice to see you again, Rainie."                                                                 │ joy     │
│ "But I am not someone you can follow carelessly," he said with a serious tone and I can feel the      │ fear    │
│ goosebumps all over my body. The first thing that came into my mind is to escape but I don't want to  │         │
│ be seen as a coward. Not by him.                                                                      │         │
│ I can see a huge, barren field, with crows, ravens and vultures flying around it. The sky was red     │ fear    │
│ because of the setting sun and it heightened the eerie scenery. I can also hear voices, some of them  │         │
│ are angry and the others are wailing their hearts out. Standing in the middle of the field was an old │         │
│ man holding his weapon—a bardiche.                                                                    │         │
│ "Hey." I stepped forward and slapped him hard. "Call me anything but stupid because I am not." After  │ anger   │
│ getting my fan from my side, I pointed it at him. "Remember that and I hope it won't fade away from   │         │
│ your memories."                                                                                       │         │
│ We both went quiet after that. I don't know what happened but after a few seconds, we smiled at each  │ joy     │
│ other. And this time, I know that something has changed between us.                                   │         │
│ "Thanks, Naomi," he said. "I owe you one."                                                            │ joy     │
│ "Maybe coming here is a wrong decision after all. Damn it, old fart. This is all your fault."         │ anger   │
│ "So you heard it, huh?" he calmly said and his tone was bone-chilling.                                │ fear    │
│ "Thank you," I said and he smiled in return.                                                          │ joy     │
│ I can't remember anything from my parents but my mother's favorite line: A person's memories can be   │ sadness │
│ seen clearly through his eyes. She's right, because right now, Mitsuo's memories flooded my own mind. │         │
│ A sea of blood, hundreds of corpses, people's cry all over the place, and him in the middle of all of │         │
│ that. Using all of my remaining mental strength, I shifted my gaze from his eyes to his wound, and I  │         │
│ fell on my knees, panting heavily.                                                                    │         │
│ "Yes and it's creepy."                                                                                │ fear    │
│ "Okay lang ako! Ano ba kayo?"                                                                         │ anger   │
│ "Nasira ang iyong tibialis anterior."                                                                 │ fear    │
│ "Ha? Saan 'yun? Ay basta, ayos lang ako. Habulin na lang natin 'yung kriminal at baka makatakas       │ anger   │
│ siya!"                                                

## Function: `convert_to_lowercase`

The `convert_to_lowercase` function converts all text in the 'sentence' column of a DataFrame to lowercase.

It ensures uniformity in text data by transforming all characters to lowercase, which is a key preprocessing step in natural language processing (NLP) tasks.

### Code

In [97]:
def convert_to_lowercase(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].str.lower()
        print("Sentence has been converted to lowercase.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

convert_to_lowercase(data)
print_table(data, title="Data After Lowercase Conversion")

Sentence has been converted to lowercase.


                                          Data After Lowercase Conversion                                          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ "your tibialis anterior is damaged."                                                                  │ fear    │
│ "sadly, he's in the west."                                                                            │ sadness │
│ "sorry."                                                                                              │ sadness │
│ "it’s just a joke."                                                                                   │ joy     │
│ "it's nice to see you again, rainie."                                                                 │ joy     │
│ "but i am not someone you can follow carelessly," he said with a serious tone and i can feel the      │ fear    │
│ goosebumps all over my body. the first thing that came into my mind is to escape but i don't want to  │         │
│ be seen as a coward. not by him.                                                                      │         │
│ i can see a huge, barren field, with crows, ravens and vultures flying around it. the sky was red     │ fear    │
│ because of the setting sun and it heightened the eerie scenery. i can also hear voices, some of them  │         │
│ are angry and the others are wailing their hearts out. standing in the middle of the field was an old │         │
│ man holding his weapon—a bardiche.                                                                    │         │
│ "hey." i stepped forward and slapped him hard. "call me anything but stupid because i am not." after  │ anger   │
│ getting my fan from my side, i pointed it at him. "remember that and i hope it won't fade away from   │         │
│ your memories."                                                                                       │         │
│ we both went quiet after that. i don't know what happened but after a few seconds, we smiled at each  │ joy     │
│ other. and this time, i know that something has changed between us.                                   │         │
│ "thanks, naomi," he said. "i owe you one."                                                            │ joy     │
│ "maybe coming here is a wrong decision after all. damn it, old fart. this is all your fault."         │ anger   │
│ "so you heard it, huh?" he calmly said and his tone was bone-chilling.                                │ fear    │
│ "thank you," i said and he smiled in return.                                                          │ joy     │
│ i can't remember anything from my parents but my mother's favorite line: a person's memories can be   │ sadness │
│ seen clearly through his eyes. she's right, because right now, mitsuo's memories flooded my own mind. │         │
│ a sea of blood, hundreds of corpses, people's cry all over the place, and him in the middle of all of │         │
│ that. using all of my remaining mental strength, i shifted my gaze from his eyes to his wound, and i  │         │
│ fell on my knees, panting heavily.                                                                    │         │
│ "yes and it's creepy."                                                                                │ fear    │
│ "okay lang ako! ano ba kayo?"                                                                         │ anger   │
│ "nasira ang iyong tibialis anterior."                                                                 │ fear    │
│ "ha? saan 'yun? ay basta, ayos lang ako. habulin na lang natin 'yung kriminal at baka makatakas       │ anger   │
│ siya!"                                                

## Function: `remove_punctuation`

The `remove_punctuation` function removes punctuation from all text in the 'sentence' column of a DataFrame.

It ensures that only the meaningful words remain, free of any punctuation that could interfere with subsequent text processing steps.

### Code

In [98]:
def remove_punctuation(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
        print("Punctuation has been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_punctuation(data)
print_table(data, title="Data After Punctuation Removal")



Punctuation has been removed.


                                          Data After Punctuation Removal                                           
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ your tibialis anterior is damaged                                                                     │ fear    │
│ sadly hes in the west                                                                                 │ sadness │
│ sorry                                                                                                 │ sadness │
│ it’s just a joke                                                                                      │ joy     │
│ its nice to see you again rainie                                                                      │ joy     │
│ but i am not someone you can follow carelessly he said with a serious tone and i can feel the         │ fear    │
│ goosebumps all over my body the first thing that came into my mind is to escape but i dont want to be │         │
│ seen as a coward not by him                                                                           │         │
│ i can see a huge barren field with crows ravens and vultures flying around it the sky was red because │ fear    │
│ of the setting sun and it heightened the eerie scenery i can also hear voices some of them are angry  │         │
│ and the others are wailing their hearts out standing in the middle of the field was an old man        │         │
│ holding his weapon—a bardiche                                                                         │         │
│ hey i stepped forward and slapped him hard call me anything but stupid because i am not after getting │ anger   │
│ my fan from my side i pointed it at him remember that and i hope it wont fade away from your memories │         │
│ we both went quiet after that i dont know what happened but after a few seconds we smiled at each     │ joy     │
│ other and this time i know that something has changed between us                                      │         │
│ thanks naomi he said i owe you one                                                                    │ joy     │
│ maybe coming here is a wrong decision after all damn it old fart this is all your fault               │ anger   │
│ so you heard it huh he calmly said and his tone was bonechilling                                      │ fear    │
│ thank you i said and he smiled in return                                                              │ joy     │
│ i cant remember anything from my parents but my mothers favorite line a persons memories can be seen  │ sadness │
│ clearly through his eyes shes right because right now mitsuos memories flooded my own mind a sea of   │         │
│ blood hundreds of corpses peoples cry all over the place and him in the middle of all of that using   │         │
│ all of my remaining mental strength i shifted my gaze from his eyes to his wound and i fell on my     │         │
│ knees panting heavily                                                                                 │         │
│ yes and its creepy                                                                                    │ fear    │
│ okay lang ako ano ba kayo                                                                             │ anger   │
│ nasira ang iyong tibialis anterior                                                                    │ fear    │
│ ha saan yun ay basta ayos lang ako habulin na lang natin yung kriminal at baka makatakas siya         │ anger   │
│ nakakalungkot nasa kanluran siya                                                                      │ sadness │
│ tsk kailangan kong magtago alam na ng mga pulis na ako

## Function: `remove_numbers`

The `remove_numbers` function removes numerical digits from all text in the 'sentence' column of a DataFrame.

It ensures that the text data is free from irrelevant numerical characters that could distort the analysis.

### Code

In [99]:
def remove_numbers(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].str.replace(r'\d+', '', regex=True)
        print("Numbers have been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_numbers(data)
print_table(data, title="Data After Numbers Removal")


Numbers have been removed.


                                            Data After Numbers Removal                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ your tibialis anterior is damaged                                                                     │ fear    │
│ sadly hes in the west                                                                                 │ sadness │
│ sorry                                                                                                 │ sadness │
│ it’s just a joke                                                                                      │ joy     │
│ its nice to see you again rainie                                                                      │ joy     │
│ but i am not someone you can follow carelessly he said with a serious tone and i can feel the         │ fear    │
│ goosebumps all over my body the first thing that came into my mind is to escape but i dont want to be │         │
│ seen as a coward not by him                                                                           │         │
│ i can see a huge barren field with crows ravens and vultures flying around it the sky was red because │ fear    │
│ of the setting sun and it heightened the eerie scenery i can also hear voices some of them are angry  │         │
│ and the others are wailing their hearts out standing in the middle of the field was an old man        │         │
│ holding his weapon—a bardiche                                                                         │         │
│ hey i stepped forward and slapped him hard call me anything but stupid because i am not after getting │ anger   │
│ my fan from my side i pointed it at him remember that and i hope it wont fade away from your memories │         │
│ we both went quiet after that i dont know what happened but after a few seconds we smiled at each     │ joy     │
│ other and this time i know that something has changed between us                                      │         │
│ thanks naomi he said i owe you one                                                                    │ joy     │
│ maybe coming here is a wrong decision after all damn it old fart this is all your fault               │ anger   │
│ so you heard it huh he calmly said and his tone was bonechilling                                      │ fear    │
│ thank you i said and he smiled in return                                                              │ joy     │
│ i cant remember anything from my parents but my mothers favorite line a persons memories can be seen  │ sadness │
│ clearly through his eyes shes right because right now mitsuos memories flooded my own mind a sea of   │         │
│ blood hundreds of corpses peoples cry all over the place and him in the middle of all of that using   │         │
│ all of my remaining mental strength i shifted my gaze from his eyes to his wound and i fell on my     │         │
│ knees panting heavily                                                                                 │         │
│ yes and its creepy                                                                                    │ fear    │
│ okay lang ako ano ba kayo                                                                             │ anger   │
│ nasira ang iyong tibialis anterior                                                                    │ fear    │
│ ha saan yun ay basta ayos lang ako habulin na lang natin yung kriminal at baka makatakas siya         │ anger   │
│ nakakalungkot nasa kanluran siya                                                                      │ sadness │
│ tsk kailangan kong magtago alam na ng mga pulis na ako

## Function: `tokenize_sentences`

The `tokenize_sentences` function tokenizes each sentence in the 'sentence' column of a DataFrame into individual words.

It breaks down sentences into individual words or tokens, facilitating analysis, standardizing input, and enabling various natural language processing tasks.

### Code

In [100]:
def tokenize_sentences(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda x: word_tokenize(x))
        print("Sentences have been tokenized.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

tokenize_sentences(data)
print_table(data, title="Data After Tokenization")

Sentences have been tokenized.


                                              Data After Tokenization                                              
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ["your", "tibialis", "anterior", "is", "damaged"]                                                     │ fear    │
│ ["sadly", "hes", "in", "the", "west"]                                                                 │ sadness │
│ ["sorry"]                                                                                             │ sadness │
│ ["it", "’", "s", "just", "a", "joke"]                                                                 │ joy     │
│ ["its", "nice", "to", "see", "you", "again", "rainie"]                                                │ joy     │
│ ["but", "i", "am", "not", "someone", "you", "can", "follow", "carelessly", "he", "said", "with", "a", │ fear    │
│ "serious", "tone", "and", "i", "can", "feel", "the", "goosebumps", "all", "over", "my", "body",       │         │
│ "the", "first", "thing", "that", "came", "into", "my", "mind", "is", "to", "escape", "but", "i",      │         │
│ "dont", "want", "to", "be", "seen", "as", "a", "coward", "not", "by", "him"]                          │         │
│ ["i", "can", "see", "a", "huge", "barren", "field", "with", "crows", "ravens", "and", "vultures",     │ fear    │
│ "flying", "around", "it", "the", "sky", "was", "red", "because", "of", "the", "setting", "sun",       │         │
│ "and", "it", "heightened", "the", "eerie", "scenery", "i", "can", "also", "hear", "voices", "some",   │         │
│ "of", "them", "are", "angry", "and", "the", "others", "are", "wailing", "their", "hearts", "out",     │         │
│ "standing", "in", "the", "middle", "of", "the", "field", "was", "an", "old", "man", "holding", "his", │         │
│ "weapon—a", "bardiche"]                                                                               │         │
│ ["hey", "i", "stepped", "forward", "and", "slapped", "him", "hard", "call", "me", "anything", "but",  │ anger   │
│ "stupid", "because", "i", "am", "not", "after", "getting", "my", "fan", "from", "my", "side", "i",    │         │
│ "pointed", "it", "at", "him", "remember", "that", "and", "i", "hope", "it", "wont", "fade", "away",   │         │
│ "from", "your", "memories"]                                                                           │         │
│ ["we", "both", "went", "quiet", "after", "that", "i", "dont", "know", "what", "happened", "but",      │ joy     │
│ "after", "a", "few", "seconds", "we", "smiled", "at", "each", "other", "and", "this", "time", "i",    │         │
│ "know", "that", "something", "has", "changed", "between", "us"]                                       │         │
│ ["thanks", "naomi", "he", "said", "i", "owe", "you", "one"]                                           │ joy     │
│ ["maybe", "coming", "here", "is", "a", "wrong", "decision", "after", "all", "damn", "it", "old",      │ anger   │
│ "fart", "this", "is", "all", "your", "fault"]                                                         │         │
│ ["so", "you", "heard", "it", "huh", "he", "calmly", "said", "and", "his", "tone", "was",              │ fear    │
│ "bonechilling"]                                                                                       │         │
│ ["thank", "you", "i", "said", "and", "he", "smiled", "in", "return"]                                  │ joy     │
│ ["i", "cant", "remember", "anything", "from", "my", "parents", "but", "my", "mothers", "favorite",    │ sadness │
│ "line", "a", "persons", "memories", "can", "be", "seen", "clearly", "through", "his", "eyes", "shes", │         │
│ "right", "because", "right", "now", "mitsuos", "memori

## Function: `remove_stopwords`

The `remove_stopwords` function removes stopwords from each tokenized sentence in the 'sentence' column of a DataFrame.

Stopwords are common words that usually add little meaning to a sentence and are often filtered out in text processing to enhance the focus on more meaningful words.

### Code

In [101]:
def remove_stopwords(data):
    if 'sentence' in data.columns:
        english_stopwords = set(stopwords.stopwords('english'))
        all_stopwords = english_stopwords.union(set(filipino_stopwords))
        
        data['sentence'] = data['sentence'].apply(lambda tokens: [word for word in tokens if word.lower() not in all_stopwords])
        print("Stopwords have been removed.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

remove_stopwords(data)
print_table(data, title="Data After Stopwords Removal")

Stopwords have been removed.


                                           Data After Stopwords Removal                                            
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ["your", "tibialis", "anterior", "is", "damaged"]                                                     │ fear    │
│ ["sadly", "hes", "in", "the", "west"]                                                                 │ sadness │
│ ["sorry"]                                                                                             │ sadness │
│ ["it", "’", "s", "just", "a", "joke"]                                                                 │ joy     │
│ ["its", "nice", "to", "see", "you", "again", "rainie"]                                                │ joy     │
│ ["but", "i", "not", "someone", "you", "can", "follow", "carelessly", "he", "said", "with", "a",       │ fear    │
│ "serious", "tone", "and", "i", "can", "feel", "the", "goosebumps", "all", "over", "my", "body",       │         │
│ "the", "first", "thing", "that", "came", "into", "my", "mind", "is", "to", "escape", "but", "i",      │         │
│ "dont", "want", "to", "be", "seen", "as", "a", "coward", "not", "by", "him"]                          │         │
│ ["i", "can", "see", "a", "huge", "barren", "field", "with", "crows", "ravens", "and", "vultures",     │ fear    │
│ "flying", "around", "it", "the", "sky", "was", "red", "because", "of", "the", "setting", "sun",       │         │
│ "and", "it", "heightened", "the", "eerie", "scenery", "i", "can", "also", "hear", "voices", "some",   │         │
│ "of", "them", "are", "angry", "and", "the", "others", "are", "wailing", "their", "hearts", "out",     │         │
│ "standing", "in", "the", "middle", "of", "the", "field", "was", "an", "old", "man", "holding", "his", │         │
│ "weapon—a", "bardiche"]                                                                               │         │
│ ["hey", "i", "stepped", "forward", "and", "slapped", "him", "hard", "call", "me", "anything", "but",  │ anger   │
│ "stupid", "because", "i", "not", "after", "getting", "my", "fan", "from", "my", "side", "i",          │         │
│ "pointed", "it", "him", "remember", "that", "and", "i", "hope", "it", "wont", "fade", "away", "from", │         │
│ "your", "memories"]                                                                                   │         │
│ ["we", "both", "went", "quiet", "after", "that", "i", "dont", "know", "what", "happened", "but",      │ joy     │
│ "after", "a", "few", "seconds", "we", "smiled", "each", "other", "and", "this", "time", "i", "know",  │         │
│ "that", "something", "has", "changed", "between", "us"]                                               │         │
│ ["thanks", "naomi", "he", "said", "i", "owe", "you", "one"]                                           │ joy     │
│ ["maybe", "coming", "here", "is", "a", "wrong", "decision", "after", "all", "damn", "it", "old",      │ anger   │
│ "fart", "this", "is", "all", "your", "fault"]                                                         │         │
│ ["so", "you", "heard", "it", "huh", "he", "calmly", "said", "and", "his", "tone", "was",              │ fear    │
│ "bonechilling"]                                                                                       │         │
│ ["thank", "you", "i", "said", "and", "he", "smiled", "in", "return"]                                  │ joy     │
│ ["i", "cant", "remember", "anything", "from", "my", "parents", "but", "my", "mothers", "favorite",    │ sadness │
│ "line", "a", "persons", "memories", "can", "be", "seen", "clearly", "through", "his", "eyes", "shes", │         │
│ "right", "because", "right", "now", "mitsuos", "memori

## Function: `lemmatize_tokens`

The `lemmatize_tokens` function lemmatizes each token in the 'sentence' column of a DataFrame using spaCy's lemmatization.

it reduces words to their base or dictionary form (lemmas), which helps in standardizing the text by consolidating different inflections of a word into a single representation, thereby improving the performance of natural language processing tasks such as classification and clustering.

### Code

In [102]:
def lemmatize_filo(data):
    with open('../data/filipino_lemmatizer.json', 'r', encoding='utf-8') as json_file:
        lemma_dict = json.load(json_file)

    token_to_lemma = {}
    for lemma, tokenval in lemma_dict['lemma_dict'].items():
        for token in tokenval:
            token_to_lemma[token] = lemma

    for index, row in data.iterrows():
        tokens = row['sentence']
        
        if all(len(token) == 1 for token in tokens):
            tokens = ''.join(tokens).split()
        
        updated_tokens = [token_to_lemma.get(token, token) for token in tokens]
        data.at[index, 'sentence'] = updated_tokens

lemmatize_filo(data)
print_table(data, title="Data After Lemmatization in Filipino")

                                       Data After Lemmatization in Filipino                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ["your", "tibialis", "anterior", "is", "damaged"]                                                     │ fear    │
│ ["sadly", "hes", "in", "the", "west"]                                                                 │ sadness │
│ ["sorry"]                                                                                             │ sadness │
│ ["it", "’", "s", "just", "a", "joke"]                                                                 │ joy     │
│ ["its", "nice", "to", "see", "you", "again", "rainie"]                                                │ joy     │
│ ["but", "i", "not", "someone", "you", "can", "follow", "carelessly", "he", "said", "with", "a",       │ fear    │
│ "serious", "tone", "and", "i", "can", "feel", "the", "goosebumps", "all", "over", "my", "body",       │         │
│ "the", "first", "thing", "that", "came", "into", "my", "mind", "is", "to", "escape", "but", "i",      │         │
│ "dont", "want", "to", "be", "seen", "as", "a", "coward", "not", "by", "him"]                          │         │
│ ["i", "can", "see", "a", "huge", "barren", "field", "with", "crows", "ravens", "and", "vultures",     │ fear    │
│ "flying", "around", "it", "the", "sky", "was", "red", "because", "of", "the", "setting", "sun",       │         │
│ "and", "it", "heightened", "the", "eerie", "scenery", "i", "can", "also", "hear", "voices", "some",   │         │
│ "of", "them", "are", "angry", "and", "the", "others", "are", "wailing", "their", "hearts", "out",     │         │
│ "standing", "in", "the", "middle", "of", "the", "field", "was", "an", "old", "man", "holding", "his", │         │
│ "weapon—a", "bardiche"]                                                                               │         │
│ ["hey", "i", "stepped", "forward", "and", "slapped", "him", "hard", "call", "me", "anything", "but",  │ anger   │
│ "stupid", "because", "i", "not", "after", "getting", "my", "fan", "from", "my", "side", "i",          │         │
│ "pointed", "it", "him", "remember", "that", "and", "i", "hope", "it", "wont", "fade", "away", "from", │         │
│ "your", "memories"]                                                                                   │         │
│ ["we", "both", "went", "quiet", "after", "that", "i", "dont", "know", "what", "happened", "but",      │ joy     │
│ "after", "a", "few", "seconds", "we", "smiled", "each", "other", "and", "this", "time", "i", "know",  │         │
│ "that", "something", "has", "changed", "between", "us"]                                               │         │
│ ["thanks", "naomi", "he", "said", "i", "owe", "you", "one"]                                           │ joy     │
│ ["maybe", "coming", "here", "is", "a", "wrong", "decision", "after", "all", "damn", "it", "old",      │ anger   │
│ "fart", "this", "is", "all", "your", "fault"]                                                         │         │
│ ["so", "you", "heard", "it", "huh", "he", "calmly", "said", "and", "his", "tone", "was",              │ fear    │
│ "bonechilling"]                                                                                       │         │
│ ["thank", "you", "i", "said", "and", "he", "smiled", "in", "return"]                                  │ joy     │
│ ["i", "cant", "remember", "anything", "from", "my", "parents", "but", "my", "mothers", "favorite",    │ sadness │
│ "line", "a", "persons", "memories", "can", "be", "seen", "clearly", "through", "his", "eyes", "shes", │         │
│ "right", "because", "right", "now", "mitsuos", "memori

In [103]:
def lemmatize_eng(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda tokens: [nlp(token)[0].lemma_ for token in tokens])
        print("Tokens have been lemmatized.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

lemmatize_eng(data)
print_table(data, title="Data After Lemmatization in English")

Tokens have been lemmatized.


                                        Data After Lemmatization in English                                        
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ ["your", "tibialis", "anterior", "be", "damage"]                                                      │ fear    │
│ ["sadly", "he", "in", "the", "west"]                                                                  │ sadness │
│ ["sorry"]                                                                                             │ sadness │
│ ["it", """, "s", "just", "a", "joke"]                                                                 │ joy     │
│ ["its", "nice", "to", "see", "you", "again", "rainie"]                                                │ joy     │
│ ["but", "I", "not", "someone", "you", "can", "follow", "carelessly", "he", "say", "with", "a",        │ fear    │
│ "serious", "tone", "and", "I", "can", "feel", "the", "goosebump", "all", "over", "my", "body", "the", │         │
│ "first", "thing", "that", "came", "into", "my", "mind", "be", "to", "escape", "but", "I", "do",       │         │
│ "want", "to", "be", "see", "as", "a", "coward", "not", "by", "he"]                                    │         │
│ ["I", "can", "see", "a", "huge", "barren", "field", "with", "crow", "ravens", "and", "vulture",       │ fear    │
│ "fly", "around", "it", "the", "sky", "be", "red", "because", "of", "the", "set", "sun", "and", "it",  │         │
│ "heighten", "the", "eerie", "scenery", "I", "can", "also", "hear", "voice", "some", "of", "they",     │         │
│ "be", "angry", "and", "the", "other", "be", "wail", "their", "heart", "out", "stand", "in", "the",    │         │
│ "middle", "of", "the", "field", "be", "an", "old", "man", "holding", "his", "weapon", "bardiche"]     │         │
│ ["hey", "I", "step", "forward", "and", "slap", "he", "hard", "call", "I", "anything", "but",          │ anger   │
│ "stupid", "because", "I", "not", "after", "get", "my", "fan", "from", "my", "side", "I", "point",     │         │
│ "it", "he", "remember", "that", "and", "I", "hope", "it", "wo", "fade", "away", "from", "your",       │         │
│ "memory"]                                                                                             │         │
│ ["we", "both", "go", "quiet", "after", "that", "I", "do", "know", "what", "happen", "but", "after",   │ joy     │
│ "a", "few", "second", "we", "smile", "each", "other", "and", "this", "time", "I", "know", "that",     │         │
│ "something", "have", "change", "between", "we"]                                                       │         │
│ ["thank", "naomi", "he", "say", "I", "owe", "you", "one"]                                             │ joy     │
│ ["maybe", "come", "here", "be", "a", "wrong", "decision", "after", "all", "damn", "it", "old",        │ anger   │
│ "fart", "this", "be", "all", "your", "fault"]                                                         │         │
│ ["so", "you", "hear", "it", "huh", "he", "calmly", "say", "and", "his", "tone", "be", "bonechilling"] │ fear    │
│ ["thank", "you", "I", "say", "and", "he", "smile", "in", "return"]                                    │ joy     │
│ ["I", "ca", "remember", "anything", "from", "my", "parent", "but", "my", "mother", "favorite",        │ sadness │
│ "line", "a", "person", "memory", "can", "be", "see", "clearly", "through", "his", "eye", "she",       │         │
│ "right", "because", "right", "now", "mitsuos", "memory", "flood", "my", "own", "mind", "a", "sea",    │         │
│ "of", "blood", "hundred", "of", "corpse", "peoples", "cry", "all", "over", "the", "place", "and",     │         │
│ "he", "in", "the", "middle", "of", "all", "of", "that"

## Function: `join_tokens`

The `join_tokens` function joins tokens in each list within the 'sentence' column of a DataFrame back into single sentences.

it reconstructs the original text format after various transformations, enabling further analysis or modeling tasks that require complete sentences, such as text classification, sentiment analysis, or readability assessments.

### Code

In [104]:
def join_tokens(data):
    if 'sentence' in data.columns:
        data['sentence'] = data['sentence'].apply(lambda tokens: ' '.join(tokens))
        print("Tokens have been joined back into sentences.")
    else:
        print("Column 'sentence' not found in the DataFrame.")

join_tokens(data)
print_table(data, title="Data After Joining Tokens")

Tokens have been joined back into sentences.


                                             Data After Joining Tokens                                             
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┓
┃ sentence                                                                                              ┃ emotion ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━┩
│ your tibialis anterior be damage                                                                      │ fear    │
│ sadly he in the west                                                                                  │ sadness │
│ sorry                                                                                                 │ sadness │
│ it ' s just a joke                                                                                    │ joy     │
│ its nice to see you again rainie                                                                      │ joy     │
│ but I not someone you can follow carelessly he say with a serious tone and I can feel the goosebump   │ fear    │
│ all over my body the first thing that came into my mind be to escape but I do want to be see as a     │         │
│ coward not by he                                                                                      │         │
│ I can see a huge barren field with crow ravens and vulture fly around it the sky be red because of    │ fear    │
│ the set sun and it heighten the eerie scenery I can also hear voice some of they be angry and the     │         │
│ other be wail their heart out stand in the middle of the field be an old man holding his weapon       │         │
│ bardiche                                                                                              │         │
│ hey I step forward and slap he hard call I anything but stupid because I not after get my fan from my │ anger   │
│ side I point it he remember that and I hope it wo fade away from your memory                          │         │
│ we both go quiet after that I do know what happen but after a few second we smile each other and this │ joy     │
│ time I know that something have change between we                                                     │         │
│ thank naomi he say I owe you one                                                                      │ joy     │
│ maybe come here be a wrong decision after all damn it old fart this be all your fault                 │ anger   │
│ so you hear it huh he calmly say and his tone be bonechilling                                         │ fear    │
│ thank you I say and he smile in return                                                                │ joy     │
│ I ca remember anything from my parent but my mother favorite line a person memory can be see clearly  │ sadness │
│ through his eye she right because right now mitsuos memory flood my own mind a sea of blood hundred   │         │
│ of corpse peoples cry all over the place and he in the middle of all of that use all of my remain     │         │
│ mental strength I shift my gaze from his eye to his wound and I fell on my knees pant heavily         │         │
│ yes and its creepy                                                                                    │ fear    │
│ okay lang ba kayo                                                                                     │ anger   │
│ sira tibialis anterior                                                                                │ fear    │
│ ha yun basta ayos lang habol lang natin yung kriminal baka takas                                      │ anger   │
│ lungkot nasa kanluran                                                                                 │ sadness │
│ tsk tago alam pulis patay hayop yun noon kriminal                                                     │ fear    │
│ hmm bpm tiyak sabik sabay ngiti                       

## Function: `vectorize_with_tfidf`

The `vectorize_with_tfidf` function performs TF-IDF vectorization on the 'sentence' column of a DataFrame.

**TF-IDF** is a statistical measure used to evaluate the importance of a word in a document relative to a collection (or corpus) of documents. It combines two key concepts:

1. **Term Frequency (TF)**: Measures how frequently a word appears in a single document. Words that appear frequently within a document are considered more important for that document.
   - Formula: `TF = (Number of times a term appears in a document) / (Total number of terms in the document)`

2. **Inverse Document Frequency (IDF)**: Measures how important a word is across the entire dataset. Words that appear in many documents (common words) are less important, while rare words are more important.
   - Formula: `IDF = log(Total number of documents / Number of documents containing the term)`

The **TF-IDF score** is calculated by multiplying TF and IDF for each word, providing a numerical value representing the word's significance within a document relative to the entire corpus.

### Code

In [105]:
def vectorize_with_tfidf(data):
    if 'sentence' in data.columns:
        vectorizer = TfidfVectorizer()

        tfidf_matrix = vectorizer.fit_transform(data['sentence'])

        print("TF-IDF Vectorization complete.")
        return tfidf_matrix, vectorizer
    else:
        print("Column 'sentence' not found in the DataFrame.")

tfidf_matrix, vectorizer = vectorize_with_tfidf(data)
feature_names = vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df['emotion'] = data['emotion'].values

# Save the DataFrame to a CSV file
tfidf_df.to_csv('tfidf_vectorized_data.csv', index=False)

print(tfidf_df.head())

TF-IDF Vectorization complete.
   about  after     again  akalaing  akane  akemi  akong  alam  alert  all  \
0    0.0    0.0  0.000000       0.0    0.0    0.0    0.0   0.0    0.0  0.0   
1    0.0    0.0  0.000000       0.0    0.0    0.0    0.0   0.0    0.0  0.0   
2    0.0    0.0  0.000000       0.0    0.0    0.0    0.0   0.0    0.0  0.0   
3    0.0    0.0  0.000000       0.0    0.0    0.0    0.0   0.0    0.0  0.0   
4    0.0    0.0  0.460933       0.0    0.0    0.0    0.0   0.0    0.0  0.0   

   ...  year  yell  yelo  yes       you  young      your  yun  yung  emotion  
0  ...   0.0   0.0   0.0  0.0  0.000000    0.0  0.393362  0.0   0.0     fear  
1  ...   0.0   0.0   0.0  0.0  0.000000    0.0  0.000000  0.0   0.0  sadness  
2  ...   0.0   0.0   0.0  0.0  0.000000    0.0  0.000000  0.0   0.0  sadness  
3  ...   0.0   0.0   0.0  0.0  0.000000    0.0  0.000000  0.0   0.0      joy  
4  ...   0.0   0.0   0.0  0.0  0.278796    0.0  0.000000  0.0   0.0      joy  

[5 rows x 356 columns]


## Save a Model

In [106]:
import pickle
import os

def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)

    print(f"Model saved to {filename}")

current_dir = os.getcwd()

models_folder = os.path.join(current_dir, "..", "models")
if not os.path.exists(models_folder):
    os.makedirs(models_folder)

## Save the TF-IDF Vectorizer Model

In [107]:
save_model(vectorizer, os.path.join(models_folder, "tfidf_vectorizer_model.pkl"))

Model saved to c:\xampp\htdocs\puplms\Emotion-Recognition-System\backend\preprocessing\..\models\tfidf_vectorizer_model.pkl


## Function: `RandomOverSampler` and `RandomUnderSampler`

In [108]:
X = tfidf_df.drop(columns=['emotion'])
y = tfidf_df['emotion']

print("Class distribution before resampling:")
print(y.value_counts())
print("\nData before resampling:")
print(tfidf_df)

def resample_data(X, y):
    oversampler = RandomOverSampler(random_state=42)
    X_over, y_over = oversampler.fit_resample(X, y)
    
    print(f"\nOversampled class distribution:\n{y_over.value_counts()}")

    undersampler = RandomUnderSampler(random_state=42)
    X_resampled, y_resampled = undersampler.fit_resample(X_over, y_over)
    
    print(f"\nFinal resampled class distribution:\n{y_resampled.value_counts()}")
    
    return X_resampled, y_resampled

X_resampled, y_resampled = resample_data(X, y)

resampled_df = pd.concat([pd.DataFrame(X_resampled, columns=feature_names), pd.DataFrame(y_resampled, columns=['emotion'])], axis=1)

resampled_df.to_csv('../data/resampled_df.csv', index=False)

print("\nData after resampling:")
print(resampled_df)

Class distribution before resampling:
emotion
joy        21
fear       16
anger      10
sadness     7
Name: count, dtype: int64

Data before resampling:
       about     after     again  akalaing     akane     akemi    akong  \
0   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000   
1   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000   
2   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000   
3   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000   
4   0.000000  0.000000  0.460933  0.000000  0.000000  0.000000  0.00000   
5   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000   
6   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000   
7   0.000000  0.142026  0.000000  0.000000  0.000000  0.000000  0.00000   
8   0.000000  0.288143  0.000000  0.000000  0.000000  0.000000  0.00000   
9   0.000000  0.000000  0.000000  0.000000  0.000000  0.000000  0.00000   
10  0.000000  0.202334

## FUNCTION: `TRAINING THE MODEL`

In [109]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

# LOAD RESAMPLED DATASET
file_path = '../data/resampled_df.csv'
data = pd.read_csv(file_path)

# SPLIT DATA INTO FEATURES (X) AND TARGET (y)
X = data.drop(columns=['emotion'])
y = data['emotion']

# SPLIT THE DATA INTO TRAINING AND TESTING SETS
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# CREATE AND TRAIN THE LOGISTIC REGRESSION MODEL (GLM)
glm = LogisticRegression(max_iter=1000)  # Increased max_iter to ensure convergence
glm.fit(X_train, y_train)

# PREDICT THE TEST SET
y_pred = glm.predict(X_test)

# GENERATE CLASSIFICATION REPORT (PRECISION, RECALL, F1-SCORE)
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

# OPTIONAL: SAVE THE TRAINED MODEL
import pickle
import os

def save_model(model, filename):
    with open(filename, 'wb') as f:
        pickle.dump(model, f)
    print(f"Model saved to {filename}")

models_folder = os.path.join(os.getcwd(), "..", "models")
if not os.path.exists(models_folder):
    os.makedirs(models_folder)

save_model(glm, os.path.join(models_folder, "emotion_recognition_glm_model.pkl"))

Classification Report:
              precision    recall  f1-score   support

       anger       1.00      0.80      0.89         5
        fear       0.60      0.75      0.67         4
         joy       0.75      0.75      0.75         4
     sadness       1.00      1.00      1.00         4

    accuracy                           0.82        17
   macro avg       0.84      0.82      0.83        17
weighted avg       0.85      0.82      0.83        17

Model saved to c:\xampp\htdocs\puplms\Emotion-Recognition-System\backend\preprocessing\..\models\emotion_recognition_glm_model.pkl
